<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/04_Regularization_Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from threading import active_count

from tensorflow.python.keras.layers.core import Activation
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import cifar10


# Device setting

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Load data

In [3]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 11s 0us/step


# Normalize

In [4]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Initialize model

In [5]:
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)),
     layers.Conv2D(32, 3, padding='valid', activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64, 3, activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(128, 3, activation='relu'),
     layers.Flatten(),
     layers.Dense(64,activation='relu'),
     layers.Dense(10),

    ]
)

In [6]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 15, 15, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 flatten (Flatten)           (None, 2048)              0

In [7]:
def my_model():
  with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32, 32, 3))
      x = layers.Conv2D(
          32, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(inputs)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(
          64, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(
          128, 3, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      
      x = layers.Flatten()(x)
      x = layers.Dense(
          64, activation="relu", kernel_regularizer=regularizers.l2(0.01))(x)
      x = layers.Dropout(0.5)(x)
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model

In [8]:
model = my_model()

In [9]:
print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_3 (Conv2D)           (None, 32, 32, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 tf.nn.relu (TFOpLambda)     (None, 32, 32, 32)        0         
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 16, 16, 64)        18496 

# Model compile

In [10]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics=['accuracy'],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# Train and evaluate

In [11]:
with tf.device('/device:GPU:0'):
  history = model.fit(x_train, y_train, batch_size=64, epochs=150, verbose=2)
  model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/150
782/782 - 20s - loss: 2.9899 - accuracy: 0.3240 - 20s/epoch - 25ms/step
Epoch 2/150
782/782 - 8s - loss: 1.9226 - accuracy: 0.4370 - 8s/epoch - 10ms/step
Epoch 3/150
782/782 - 8s - loss: 1.6508 - accuracy: 0.4925 - 8s/epoch - 10ms/step
Epoch 4/150
782/782 - 8s - loss: 1.5308 - accuracy: 0.5176 - 8s/epoch - 10ms/step
Epoch 5/150
782/782 - 8s - loss: 1.4683 - accuracy: 0.5349 - 8s/epoch - 10ms/step
Epoch 6/150
782/782 - 8s - loss: 1.4211 - accuracy: 0.5518 - 8s/epoch - 10ms/step
Epoch 7/150
782/782 - 8s - loss: 1.3999 - accuracy: 0.5593 - 8s/epoch - 10ms/step
Epoch 8/150
782/782 - 8s - loss: 1.3667 - accuracy: 0.5696 - 8s/epoch - 10ms/step
Epoch 9/150
782/782 - 8s - loss: 1.3477 - accuracy: 0.5763 - 8s/epoch - 10ms/step
Epoch 10/150
782/782 - 8s - loss: 1.3259 - accuracy: 0.5844 - 8s/epoch - 10ms/step
Epoch 11/150
782/782 - 8s - loss: 1.3135 - accuracy: 0.5889 - 8s/epoch - 10ms/step
Epoch 12/150
782/782 - 8s - loss: 1.2971 - accuracy: 0.5960 - 8s/epoch - 10ms/step
Epoch 13/15